In [ ]:
!nvidia-smi

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install -q transformers

Mounted at /content/gdrive
     |████████████████████████████████| 4.7 MB 25.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.7 MB/s 
     |████████████████████████████████| 101 kB 7.6 MB/s 
     |████████████████████████████████| 596 kB 7.8 MB/s 


In [2]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  javascript-common libcupsfilters1 libcupsimage2 libgs9 libgs9-common
  libijs-0.35 libjbig2dec0 libjs-jquery libkpathsea6 libpotrace0 libptexenc1
  libruby2.5 libsynctex1 libtexlua52 libtexluajit2 libzzip-0-13 lmodern
  poppler-data preview-latex-style rake ruby ruby-did-you-mean ruby-minitest
  ruby-net-telnet ruby-power-assert ruby-test-unit ruby2.5
  rubygems-integration t1utils tex-common tex-gyre texlive-base
  texlive-binaries texlive-latex-base texlive-latex-extra
  texlive-latex-recommended texlive-pictures tipa
Suggested packages:
  fonts-noto apache2 | lighttpd | httpd poppler-utils ghostscript
  fonts-japanese-mi

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [4]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import TFBertModel, TFBertForSequenceClassification, BertConfig, TFAutoModelForSequenceClassification
from transformers import glue_convert_examples_to_features
from transformers import InputExample, InputFeatures
import tensorflow as tf


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")



In [13]:
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        _x = str(_x)
        if isinstance(_x, str):
            text_a = _x
            text_b = ""
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    
    features = glue_convert_examples_to_features(
        examples, 
        tokenizer, 
        maxlen, 
        output_mode=output_mode, 
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [14]:

gdrive_path = '/content/gdrive/My Drive/ColabNotebooks/'

Test = 'test_set_text.csv'
Train = 'train_set_text.csv'

train_df = pd.read_csv(gdrive_path + Train)
test_df = pd.read_csv(gdrive_path + Test)
print(train_df.columns)
x_train, y_train = train_df['text'].values, train_df['target'].values
x_test, y_test = test_df['text'].values, test_df['target'].values
print(type(x_train))
# print(type(x_train[0]))
print(type(x_train[0]))

Index(['Unnamed: 0', 'text', 'label', 'target'], dtype='object')
<class 'numpy.ndarray'>
<class 'str'>


In [15]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=256)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_test, y_test, maxlen=256)


  0%|          | 0/8799 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


  0%|          | 0/8799 [00:00<?, ?it/s]

  0%|          | 0/2172 [00:00<?, ?it/s]

  0%|          | 0/2172 [00:00<?, ?it/s]

In [16]:
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'attention_mask: {value[0]["attention_mask"]}')
    print(f'token_type_ids: {value[0]["token_type_ids"]}')
    print(f'        target: {value[1]}')
    break

     input_ids: [  101   464 43868 11872 55693 10613 11879 72100 53493 12565   476 69779
 10314 10266 28102 10427 52780 91339 15675   479 47724 15675 34203 13265
   446 30465 10427 28970 11024 29161 12066 34769 10411 11045 86246   479
   464 43868 11872 10731 55693 10364 52780 31385   454 14363 68149 51164
 12565   451 10558 16124 40928 37212   119 10266 10731 20747 15844 10314
 10427 52780 31797 14523 38321 21742 47238   479 12066 34769 10411 11045
   464 43868 11872 10731 55693 76471 10615 51164 12565 10856 22315 27328
 23117   119 10364 74236 40966 14636 10364 34338 10427 83335   515 16511
 30196 43920 11045 47188   446 28970 21052 38321 10615 60861 10615   476
 69779 10314   479 12449 37701 93592 10427 31425 12449 37701   462 13072
 16914 10856 10364 10731 16984   476 21095 23987 13201   446 18726   464
 21797 12449 37701   479 12651 19268 10266 21052 35959 82115 48488   479
   476 21095 23987 34445 12121   119 51023 16511 10645   446 76387 51536
 74857 12565 10427   464 43868 1187

In [17]:
def build_model(model_name, config, learning_rate=3e-5):
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [18]:
train_dataset_base = train_dataset_base.batch(10)
valid_dataset_base = valid_dataset_base.batch(10)

MODEL_NAME = 'bert-base-multilingual-uncased'

# model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

labels = list(train_df['label'].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')


config = AutoConfig.from_pretrained(
    MODEL_NAME, **{
        'label2id': label2id,
        'id2label': id2label,
    })
model = build_model(MODEL_NAME, config, learning_rate=2e-5)
# r = model.fit(
    # train_dataset_base,
    # validation_data=valid_dataset_base,
    # epochs=1,
    # verbose=1)

# final_accuracy = r.history['val_accuracy']
# print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))


label2id: {'اقتصادي': 0, 'فرهنگي': 1, 'علمي': 2, 'سياسي': 3, 'ورزشي': 4, 'آموزشي': 5, 'فقه و حقوق': 6, 'بهداشتي': 7, 'تاريخي': 8, 'مذهبي': 9, 'اجتماعي': 10}
id2label: {0: 'اقتصادي', 1: 'فرهنگي', 2: 'علمي', 3: 'سياسي', 4: 'ورزشي', 5: 'آموزشي', 6: 'فقه و حقوق', 7: 'بهداشتي', 8: 'تاريخي', 9: 'مذهبي', 10: 'اجتماعي'}


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
r = model.fit(
    train_dataset_base,
    validation_data=valid_dataset_base,
    epochs=3,
    verbose=1)

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))


Epoch 1/3
880/880 [==============================] - 578s 640ms/step - loss: 0.8445 - accuracy: 0.7560 - val_loss: 0.4330 - val_accuracy: 0.8706
Epoch 2/3
880/880 [==============================] - 560s 637ms/step - loss: 0.4214 - accuracy: 0.8820 - val_loss: 0.4914 - val_accuracy: 0.8651
Epoch 3/3
880/880 [==============================] - 560s 636ms/step - loss: 0.3291 - accuracy: 0.9077 - val_loss: 0.4144 - val_accuracy: 0.8895
FINAL ACCURACY MEAN:  0.8750767310460409


In [23]:
 !jupyter nbconvert --to pdf /content/gdrive/MyDrive/ColabNotebooks2/13.ipynb

[NbConvertApp] Converting notebook /content/gdrive/MyDrive/ColabNotebooks2/13.ipynb to pdf
[NbConvertApp] Writing 75972 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 62437 bytes to /content/gdrive/MyDrive/ColabNotebooks2/13.pdf
